In [1]:
#######################################################
#### Set this value to the directory of period-suite ##
#######################################################
pathToSuite="/Users/sertoez/git-projects/period-suite/"
#######################################################

import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
precision=100; # will be OVERRIDDEN by user specified precision, under normal usage
# if increasing this precision manually, do not forget to recompute initial conditions to higher precision.
#
#
#
precisionList=[]; # if left empty, every ODE will be integrated using the same precision
paths=[]; #unless overridden we will convert this to a list of straight paths

t0=time.time();
load(pathToSuite+"current.sage")
t1=time.time();
print "Loaded file in", t1-t0, "seconds";
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precisionList)==0:
    precisionList=[10^-precision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precisionList[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIntegration took", t1-t0, "seconds in total.\n"
#print Matrix(periods).str()


# Write the periods to file
    
outputFile = open(pathToSuite+"lastPeriods",'w')

precision=-maximalError.log(10).round()
outputFile.write(str(precision)+"\n")

digits=ceil(precision/100)*100;
outputFile.write(str(digits)+"\n")

bits=ceil(log(10^digits)/log(2))+10
Rr=RealField(bits)
def print_number(num):
    re=Rr(num.real())
    im=Rr(num.imag())
    re_str=abs(re).str(digits=digits,no_sci=2)+"p"+str(digits)
    im_str=abs(im).str(digits=digits,no_sci=2)+"p"+str(digits)+"*I"
    if re.is_square():
        num_str=re_str
    else:
        num_str="-"+re_str
    if im.is_square():
        num_str=num_str+"+"+im_str
    else:
        num_str=num_str+"-"+im_str
    return num_str

numrows=periods.nrows()
numcols=periods.ncols()

prefix="Matrix(CC," +str(numrows) +","+str(numcols)+", [CC|"

numels=numrows*numcols
flat_periods=periods.list()
mat=print_number(flat_periods[0])

for i in [1..(numels-1)]:
    mat=mat+","+print_number(flat_periods[i])
    
outputFile.write(prefix+mat+"])")
outputFile.close()


Loaded file in 0.699176073074 seconds
Integrating systems 1 through 3

Integrating system number 1
	ODE number 1 took 1.46664786339 seconds to integrate
	ODE number 2 took 1.14164614677 seconds to integrate
	ODE number 3 took 0.786028146744 seconds to integrate
	ODE number 4 took 1.17527008057 seconds to integrate
	ODE number 5 took 1.22373604774 seconds to integrate
	ODE number 6 took 1.82155299187 seconds to integrate
	ODE number 7 took 1.69833302498 seconds to integrate
	ODE number 8 took 0.862948894501 seconds to integrate
	ODE number 9 took 1.13526296616 seconds to integrate
	ODE number 10 took 1.15518093109 seconds to integrate
	ODE number 11 took 1.87483406067 seconds to integrate
	ODE number 12 took 1.94773507118 seconds to integrate
	ODE number 13 took 1.54274916649 seconds to integrate
	ODE number 14 took 1.43718099594 seconds to integrate
	ODE number 15 took 1.06415200233 seconds to integrate
	ODE number 16 took 1.63332009315 seconds to integrate
	ODE number 17 took 1.727946

In [3]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 75
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 95
##
##

load(pathToSuite+"IntersectionMatrix.sage")

periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms2

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"
print "First row records the norm of the relation."
print "Second row shows if the relation continues to hold with more digits.\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms1[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms1[j], testRelations[j]
    j=j+1



Picard rank appears to be 12 

Check to see if this makes sense:

First row records the norm of the relation.
Second row shows if the relation continues to hold with more digits.

01 || ---hyperplane section---
02 || 1.41421356237310 (0, 0)
03 || 1.41421356237310 (0, 0)
04 || 1.41421356237310 (0, 0)
05 || 1.41421356237310 (0, 0)
06 || 1.41421356237310 (0, 0)
07 || 1.41421356237310 (0, 0)
08 || 1.41421356237310 (0, 0)
09 || 1.41421356237310 (0, 0)
10 || 1.41421356237310 (0, 0)
11 || 2.00000000000000 (0, 0)
12 || 2.00000000000000 (0, 0)
13 || 2.57425228670551e15 (-79313511711424240645179571025157986, 79313511711424240645179571025157986)
14 || 3.55042968253404e15 (272740671197200410811325047811280, 0)
15 || 3.33855930457005e15 (76196466136993694386376860597768859, -77489730120893785614053541474997155)
16 || 2.98692780292577e15 (-79313511711424240645179571025157986, -79313511711424240645179571025157986)
17 || 3.67316542559692e15 (0, -272740671197200410811325047811280)
18 || 3.661719016359